1) Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

        название источника;
        наименование новости;
        ссылку на новость;
        дата публикации.
        
2) Сложить собранные новости в БД

In [94]:
from lxml import html
import requests
from pymongo import MongoClient
from pprint import pprint

In [88]:
news = []

keys = ('title', 'date', 'link')
lenta_url = 'https://lenta.ru/'

request = requests.get(lenta_url)

root = html.fromstring(request.text)
root.make_links_absolute(lenta_url)

news_links = root.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                            //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                            /a/@href''')

news_text = root.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                            //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                            /a/text()''')

news_datetime = root.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                            //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                            /a/time/@datetime''')

for i in range(len(news_text)):
    news_text[i] = news_text[i].replace(u'\xa0', u' ')

for item in list(zip(news_text, news_datetime, news_links)):
    news_dict = {}
    for key, value in zip(keys, item):
        news_dict[key] = value

    news_dict['source'] = 'lenta.ru'
    news.append(news_dict)

In [95]:
pprint(news)

[{'date': ' 16:12, 27 ноября 2021',
  'link': 'https://lenta.ru/news/2021/11/27/bloomberg_speaks/',
  'source': 'lenta.ru',
  'title': 'В США увидели скрытый смысл в отправке российских военных к '
           'границе с Украиной'},
 {'date': ' 19:26, 27 ноября 2021',
  'link': 'https://lenta.ru/news/2021/11/27/britawards/',
  'source': 'lenta.ru',
  'title': 'Брайан Мэй раскритиковал культуру отмены и отказ от гендерных '
           'номинаций'},
 {'date': ' 19:23, 27 ноября 2021',
  'link': 'https://moslenta.ru/news/v-moskve-uvelichat-socialnye-vyplaty-semyam-s-detmi-27-11-2021.htm/?utm_source=from_lenta',
  'source': 'lenta.ru',
  'title': 'В Москве увеличат социальные выплаты семьям с детьми '},
 {'date': ' 19:18, 27 ноября 2021',
  'link': 'https://lenta.ru/news/2021/11/27/catchomikron/',
  'source': 'lenta.ru',
  'title': 'ЮАР поделится с другими странами образцом нового штамма '
           'коронавируса'},
 {'date': ' 18:44, 27 ноября 2021',
  'link': 'https://lenta.ru/news/2021/

In [96]:
client = MongoClient('localhost', 27017)
mongo_db = client['lentaru']
topnews = mongo_db.topnews
topnews.insert_many(news)

<img src="2021-11-27_19-45-19.png" alt="Drawing" style="width: 1000px;">